In [ ]:
from fastai.vision.all import *

In [ ]:
path = untar_data(URLs.MNIST_SAMPLE)

In [ ]:
path.ls()

(#3) [Path('/storage/data/mnist_sample/train'),Path('/storage/data/mnist_sample/labels.csv'),Path('/storage/data/mnist_sample/valid')]

In [ ]:
train3_folder = (path/'train'/'3').ls() # dirección de las imágenes
train7_folder = (path/'train'/'7').ls()
valid3_folder = (path/'valid'/'3').ls()
valid7_folder = (path/'valid'/'7').ls()

In [ ]:
train_7 = [tensor(Image.open(i)) for i in train7_folder] # crea una lista con las imágenes (puedo abrir cualquiera con show_image(train_7[]))
train_3 = [tensor(Image.open(i)) for i in train3_folder] # idem con el contenido de train3_folder

In [ ]:
def stack(folder):
    x = torch.stack(folder).float()/255
    return x

In [ ]:
ideal_3 = stack(train_3).mean(0) # crea un tensor de rango-2, donde cada pixel el el promedio 'colapsado' de las matrices que formaban el arreglo tridimensional
ideal_7 = stack(train_7).mean(0)
valid_3 = stack([tensor(Image.open(i)) for i in valid3_folder]) # crea un tensor de rango-3 (1010 matrices de 28x28, siendo cada matriz una imágen)
valid_7 = stack([tensor(Image.open(i)) for i in valid7_folder]) # idem para las imágenes del 7. Se tienen así dos sets de validación, uno para el 3 y otro para el 7

In [ ]:
train = torch.cat([stack(train_7), stack(train_3)]) # concatena los tensores de rango-3 (en la celda anterior estos dos tensores tridimensionales tenían un promedio aplicado y se habían convertido en uno bidimensional)
valid = torch.cat([valid_7, valid_3]) # concatena los tensores de rango-3
# train tiene shape (12396, 28, 28)
# valid tiene shape (2038, 28, 28)

train_x = train.view(-1, 28*28) # colapsó el tensor tridimensional en uno bidimensional. Las matrices de 28*28 ahora son vectores de largo 786
valid_x = train.view(-1, 28*28) # el set de validación se muestra también como una colección de vectores

train_y = tensor([1]*len(train_3) + [0]*len(train_7)).unsqueeze(1) # en un vector con 6131 celdas en 1 seguidas de 6265 celdas con valor 0. Son los labels de tensor train_x
valid_y = tensor([1]*len(valid3_folder) + [0]*len(valid7_folder)).unsqueeze(1) # básicamente, el vector de labels para valid_x

In [ ]:
dset = list(zip(train_x, train_y)) # genera un arreglo de tuplas (train_x,train_y) --es un arreglo donde cada vector tiene su correspondiente etiqueta--
dset_valid = list(zip(valid_x, valid_y)) # idem para el set de validación

In [ ]:
linear_model = nn.Linear(28*28,1) # reemplazo la función de init_parameters() y linear1 por esta función de python, que trae weights y bias integrados
w,b = linear_model.parameters()
w.shape, b.shape

(torch.Size([1, 784]), torch.Size([1]))

In [ ]:
class Optimizer: # reemplaza funciones antes implementadas de otra forma
    def __init__(self,params,lr):
        self.params=parameters
        self.lr = list(params)
        lr
    
    def step(self, *args, **kwargs):
        for p in self.params:
            p.data -= p.grad.data * self.lr
            
    def zero_grad(self, *args, **kwargs):
        for p in self.params:
            p.grad = None

In [ ]:
def mnist_loss(predictions, targets): # targets es como se llama a los labels. mnist_loss mide la distancia entre las predicciones y los targets
    predictions = predictions.sigmoid() # sigmoid resalta las diferencias, básicamente
    return (predictions-targets).abs().mean() # alternativa: (predictions-target).abs().mean()

In [ ]:
# torch.where(targets==1, 1-predictions, predictions).mean()

In [ ]:
def metric(prediction, target):
    return (prediction.round() == target).float().mean()

In [ ]:
dls = DataLoader(dset, batch_size=64) # crea lotes de elementos, para procesar más rápido en paralelo, en la GPU
dls_valid = DataLoader(dset_valid, batch_size=64)

In [ ]:
opt = Optimizer((w,b), lr)
opt.__init__()

def train_epoch(model):  
    for x_batch,y_batch in dls:
        pred = model(x_batch)
        loss = mnist_loss(pred,y_batch)
        loss.backward()
        print('error rate:',metric(pred,y_batch).item())
        opt.step()
        opt.zero_grad()

NameError: name 'lr' is not defined

In [ ]:
def batch_accuracy(xb,yb): # no podía hacer funcionar la función para la métrica
    preds = xb.sigmoid()
    correct = (preds>0.5) == yb
    return correct.float().mean()

In [ ]:
def validate_epoch(model):
    accs = [batch_accuracy(model(xb),yb) for xb,yb in dset_valid]
    return round(torch.stack(accs).mean().item(),4)

validate_epoch(linear1)

In [ ]:
def train_model(model, epochs):
    for i in range(epochs):
        train_epoch(model)
        print(validate_epoc(model))

In [ ]:
train_model(linear_model,20)